# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#|export

import ast
from execnb.nbio import *
from execnb.shell import *
from fastcore.utils import *
from fastcore.script import *
from pathlib import Path

The code here is based on functionality and code from [nbdev](https://github.com/AnswerDotAI/nbdev/tree/main).

For a full-fledged library creator, visit [nbdev](https://github.com/AnswerDotAI/nbdev/tree/main)!

This is just a tiny portion of the functionality, but just what I need for small pieces of code here and there (especially on remote machines).

In [ ]:
%%writefile example.ipynb
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "31bebeca-a013-4905-89d9-54e01586f70f",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "4"
      ]
     },
     "execution_count": 1,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "2+2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "5d3b74e6-ceb0-45b7-bc61-9e2047c3712a",
   "metadata": {},
   "outputs": [],
   "source": [
    "#|export\n",
    "def hello_world():\n",
    "    print('Hello, world!')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b8897db9-ea21-406e-9998-5d049202e3f3",
   "metadata": {},
   "source": [
    "Markdown cell."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

Overwriting example.ipynb


In [ ]:
nb_str = '{\n "cells": [\n  {\n   "cell_type": "code",\n   "execution_count": null,\n   "id": "31bebeca-a013-4905-89d9-54e01586f70f",\n   "metadata": {},\n   "outputs": [],\n   "source": []\n  }\n ],\n "metadata": {\n  "kernelspec": {\n   "display_name": "",\n   "name": ""\n  },\n  "language_info": {\n   "name": ""\n  }\n },\n "nbformat": 4,\n "nbformat_minor": 5\n}\n'

import codecs

with open('example.ipynb', 'w') as f:
    decoded_text = codecs.decode(repr(nb_str)[1:-1], 'unicode_escape')
    f.write(decoded_text)

In [ ]:
#|export

def strip_directives(text):
    # Split the text into lines and keep only lines that don't start with '#|'
    return '\n'.join(line for line in text.splitlines() if not line.strip().startswith('#|'))

def write_cells(cells, file):
    "Write `cells` to `file`."
    for cell in cells:
        if cell.cell_type=='code' and cell.source.strip():
            file.write(f'{strip_directives(cell.source)}')

@call_parse
def export_nb(file_name:str):
    "Export notebook with `file_name` to .py"
    fpath = Path(file_name)
    nb = read_nb(fpath)
    
    # grab the source from all the cells that have an `export` comment
    cells = L(cell for cell in nb.cells if re.match(r'#\s*\|export', cell.source))

    # find all the exported functions, to create `__all__`:
    trees = cells.map(NbCell.parsed_).concat()
    funcs = trees.filter(risinstance((ast.FunctionDef,ast.ClassDef))).attrgot('name')
    exp_funcs = [f for f in funcs if f[0]!='_']

    # write out the file
    with Path(f'{fpath.stem}.py').open('w',encoding="utf-8") as f:
        dest = '.'
        f.write(f"# Export from notebook: {fpath}\n\n")
        f.write(f"__all__ = {exp_funcs}\n\n")
        write_cells(cells, f)
        f.write('\n')

In [ ]:
export_nb('example.ipynb')

In [ ]:
cat example.py

# Export from notebook: example.ipynb

__all__ = []




In [ ]:
!rm example.ipynb example.py

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()